In [2]:
%pip install gradio torch facenet-pytorch numpy Pillow opencv-python pytorch-grad-cam
%pip install torch
%pip install facenet-pytorch
%pip install opencv-python
%pip install pytorch-gradcam
%pip install gradio
%pip install pytorch-gradcam
%pip install grad-cam

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 55.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 76.7 MB/s eta 0:00:00
ERROR: Could not find a version that satisfies the requirement pytorch-grad-cam (from versions: none)
ERROR: No matching distribution found for pytorch-grad-cam
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 59.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 61.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 87.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━

In [1]:
import gradio as gr
import torch
import torch.nn. functional as F
from facenet_pytorch import MTCNN, InceptionResnetV1
import numpy as np
from PIL import Image
import cv2
from pytorch_grad_cam import GradCAM
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget
from pytorch_grad_cam.utils.image import show_cam_on_image
import warnings
warnings.filterwarnings("ignore")

ModuleNotFoundError: No module named 'gradio'

In [4]:
DEVICE = 'cuda:0' if torch.cuda.is_available() else 'cpu'
mtcnn = MTCNN (
select_largest=False,
post_process=False,
device=DEVICE
).to(DEVICE).eval()

In [16]:
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Initialize your model
model = InceptionResnetV1(
    pretrained="vggface2",
    classify=True,
    num_classes=1,
    device=DEVICE
)

# Path to your checkpoint
checkpoint_path = "/content/resnetinceptionv1_epoch_32.pth"

# Load the checkpoint
checkpoint = torch.load(checkpoint_path, map_location=DEVICE)

# Attempt to load the model weights directly from the checkpoint
try:
    model.load_state_dict(checkpoint)
    print("Model weights loaded successfully.")
except RuntimeError as e:
    print(f"Error loading model weights: {e}")

# Move the model to the specified device and set it to evaluation mode
model.to(DEVICE)
model.eval()

RuntimeError: PytorchStreamReader failed reading zip archive: failed finding central directory

In [7]:
import os
print(os.getcwd())  # Print current working directory
print(os.listdir())  # Print the list of files in the current directory

/content
['.config', 'drive', '.ipynb_checkpoints', 'resnetinceptionv1_epoch_32.pth', 'sample_data']


In [ ]:
# !file resnetinceptionv1_epoch_32.pth

In [ ]:
# from google.colab import files
# uploaded = files.upload()

In [8]:
import os
print(os.getcwd())

/content


In [9]:
checkpoint_path = "/content/resnetinceptionv1_epoch_32.pth"

In [10]:
print(os.listdir("/content/"))

['.config', 'drive', '.ipynb_checkpoints', 'resnetinceptionv1_epoch_32.pth', 'sample_data']


In [11]:
import urllib.request

file_url = " https://github.com/davidsandberg/facenet.git"
urllib.request.urlretrieve(file_url, "resnetinceptionv1_epoch_32.pth")

('resnetinceptionv1_epoch_32.pth', <http.client.HTTPMessage at 0x7d50aea4ada0>)

In [12]:
torch.save(model.state_dict(), "resnetinceptionv1_epoch_32.pth")

In [13]:
checkpoint = torch.load("resnetinceptionv1_epoch_32.pth", map_location=torch.device('cpu'))
print(checkpoint)

OrderedDict([('conv2d_1a.conv.weight', tensor([[[[ 5.9824e-02,  1.7953e-01,  2.8926e-01],
          [-3.7600e-02,  7.0271e-02,  1.3611e-01],
          [-2.1849e-01, -8.6169e-02, -1.3002e-02]],

         [[ 2.2355e-02,  1.8516e-01,  3.1012e-01],
          [-9.2252e-02,  5.4475e-02,  1.4013e-01],
          [-3.0034e-01, -1.3172e-01, -3.9770e-02]],

         [[ 9.2837e-02,  2.0002e-01,  3.2233e-01],
          [-1.8695e-02,  7.3924e-02,  1.5134e-01],
          [-2.5090e-01, -1.1609e-01, -3.9765e-02]]],


        [[[-2.0792e-01, -9.2688e-02, -9.4617e-02],
          [ 1.2252e-02,  7.9317e-02,  8.9089e-02],
          [ 1.5763e-01,  2.2229e-01,  2.7606e-01]],

         [[-2.4610e-01, -1.2089e-01, -1.3046e-01],
          [-3.9564e-03,  7.9825e-02,  8.5580e-02],
          [ 1.2087e-01,  2.0506e-01,  2.5370e-01]],

         [[-2.1682e-01, -1.2305e-01, -1.3316e-01],
          [ 1.1448e-02,  5.7706e-02,  6.2719e-02],
          [ 2.0375e-01,  2.4775e-01,  3.0523e-01]]],


        [[[-1.3981e-03, -2.

In [14]:
def predict(input_image:Image.Image):
    """Predict the label of the input_image"""
    face = mtcnn(input_image)
    if face is None:
        raise Exception('No face detected')
    face = face.unsqueeze(0) # add the batch dimension
    face = F.interpolate(face, size=(256, 256), mode='bilinear', align_corners=False)

    # convert the face into a numpy array to be able to plot it
    prev_face = face.squeeze(0).permute(1, 2, 0).cpu().detach().int().numpy()
    prev_face = prev_face.astype('uint8')

    face = face.to(DEVICE)
    face = face.to(torch.float32)
    face = face / 255.0
    face_image_to_plot = face.squeeze(0).permute(1, 2, 0).cpu().detach().int().numpy()

    target_layers=[model.block8.branch1[-1]]
    use_cuda = True if torch.cuda.is_available() else False
    cam = GradCAM(model=model, target_layers=target_layers)
    targets = [ClassifierOutputTarget(0)]

    grayscale_cam = cam(input_tensor=face, targets=targets, eigen_smooth=True)
    grayscale_cam = grayscale_cam[0, :]
    visualization = show_cam_on_image(face_image_to_plot, grayscale_cam, use_rgb=True)
    face_with_mask = cv2.addWeighted(prev_face, 1, visualization, 0.5, 0)

    with torch.no_grad():
        output = torch.sigmoid(model(face).squeeze(0))
        prediction = "real" if output.item() < 0.5 else "fake"

        real_prediction = 1 - output.item()
        fake_prediction = output.item()

        confidences = {
            'real': real_prediction,
            'fake': fake_prediction
        }
    return confidences, face_with_mask

In [ ]:
import gradio as gr
help(gr)

In [15]:
import gradio as gr

# Fix the code by using the correct import for Gradio
interface = gr.Interface(
fn=predict,
inputs=[
gr.Image(label="Input Image", type="pil")
],
outputs=[
gr.Label(label="class"),
gr.Image(label="Face with Explainability", type="pil")
],
).launch(debug=True, share=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://8c8aaf10c7b4fe93a1.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://8c8aaf10c7b4fe93a1.gradio.live


In [ ]:
import gradio as gr
import torch
import torch.nn.functional as F
from facenet_pytorch import InceptionResnetV1
from pytorch_grad_cam import GradCAM
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget
from pytorch_grad_cam.utils.image import show_cam_on_image
import cv2
from PIL import Image
import numpy as np
import warnings
warnings.filterwarnings("ignore")

DEVICE = 'cuda:0' if torch.cuda.is_available() else 'cpu'

# Initialize your model
model = InceptionResnetV1(
    pretrained="vggface2",
    classify=True,
    num_classes=1,
    device=DEVICE
).to(DEVICE).eval()

# Path to your checkpoint
checkpoint_path = "/content/resnetinceptionv1_epoch_32.pth"

# Load the checkpoint
checkpoint = torch.load(checkpoint_path, map_location=DEVICE)
model.load_state_dict(checkpoint)
print("Model weights loaded successfully.")

def predict(input_video):
    cap = cv2.VideoCapture(input_video)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    fake_frames = 0

    for _ in range(total_frames):
        ret, frame = cap.read()
        if not ret:
            break

        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        pil_image = Image.fromarray(frame)

        # Processing the frame as you did with the input image in your original predict function
        # Note: Skip MTCNN face detection for simplification, assuming the video contains a clear face view
        # Otherwise, integrate MTCNN or another face detection method here

        face = torch.tensor(np.array(pil_image)).permute(2, 0, 1).unsqueeze(0).to(DEVICE)
        face = F.interpolate(face, size=(256, 256), mode='bilinear', align_corners=False)
        face = face / 255.0  # Normalize the face image

        target_layers = [model.block8.branch1[-1]]
        cam = GradCAM(model=model, target_layers=target_layers)
        targets = [ClassifierOutputTarget(0)]

        with torch.no_grad():
            output = torch.sigmoid(model(face).squeeze(0))
            prediction = output.item()  # Fake confidence score

        # Consider the frame as fake if prediction > 0.5
        if prediction > 0.5:
            fake_frames += 1

    cap.release()

    # Determine if the video is deepfake based on the ratio of fake frames
    video_length_in_seconds = total_frames / cap.get(cv2.CAP_PROP_FPS)
    fake_ratio = fake_frames / total_frames
    video_label = "Fake" if fake_ratio > 0.5 else "Real"  # Simple majority vote, can be adjusted

    return video_label, f"{fake_ratio*100:.2f}% of the frames were classified as fake."

# Gradio interface to take video as input
interface = gr.Interface(
    fn=predict,
    inputs=gr.Video(label="Input Video"),
    outputs=[
        gr.Textbox(label="Classification"),
        gr.Textbox(label="Details")
    ],
).launch(debug=True, share=True)


In [ ]:
# import gradio as gr
# import torch
# import torch.nn.functional as F
# from facenet_pytorch import InceptionResnetV1
# from pytorch_grad_cam import GradCAM
# from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget
# import cv2
# from PIL import Image
# import numpy as np
# import warnings
# warnings.filterwarnings("ignore")

# DEVICE = 'cuda:0' if torch.cuda.is_available() else 'cpu'

# # Initialize your model
# model = InceptionResnetV1(
#     pretrained="vggface2",
#     classify=True,
#     num_classes=1,
# ).to(DEVICE).eval()

# # Load the model weights
# checkpoint_path = "/content/resnetinceptionv1_epoch_32.pth"
# checkpoint = torch.load(checkpoint_path, map_location=DEVICE)
# model.load_state_dict(checkpoint)
# print("Model weights loaded successfully.")

# def predict(input_video):
#     cap = cv2.VideoCapture(input_video)
#     total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
#     fps = cap.get(cv2.CAP_PROP_FPS)

#     # Handle case where FPS could not be determined
#     if fps == 0:
#         print("Warning: Could not determine video FPS. Using default value of 30.")
#         fps = 30

#     fake_frames = 0

#     for _ in range(total_frames):
#         ret, frame = cap.read()
#         if not ret:
#             break

#         frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
#         pil_image = Image.fromarray(frame)
#         face = torch.tensor(np.array(pil_image)).unsqueeze(0).permute(0, 3, 1, 2).to(DEVICE)
#         face = F.interpolate(face, size=(256, 256), mode='bilinear', align_corners=False)
#         face = face / 255.0  # Normalize

#         target_layers = [model.last_bn]  # Adjust as per your model architecture
#         cam = GradCAM(model=model, target_layers=target_layers)
#         targets = None  # Assuming we want the output for the highest scoring class

#         with torch.no_grad():
#             logits = model(face)
#             predictions = torch.sigmoid(logits)
#             predicted_label = predictions.item() > 0.5
#             if predicted_label:
#                 fake_frames += 1

#     cap.release()

#     fake_ratio = fake_frames / total_frames if total_frames > 0 else 0
#     video_label = "Fake" if fake_ratio > 0.5 else "Real"
#     detail = f"{fake_ratio*100:.2f}% of the frames were classified as fake."

#     return video_label, detail

# # Gradio interface
# interface = gr.Interface(
#     fn=predict,
#     inputs=gr.Video(label="Input Video"),
#     outputs=[
#         gr.Text(label="Classification"),
#         gr.Text(label="Details")
#     ],
# ).launch(debug=True, share=True)


In [17]:
import gradio as gr
import torch
import torch.nn.functional as F
from facenet_pytorch import InceptionResnetV1
import cv2
from PIL import Image
import numpy as np
import warnings
warnings.filterwarnings("ignore")

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

# Initialize your model
model = InceptionResnetV1(pretrained="vggface2", classify=True, num_classes=1).to(DEVICE).eval()

# Load the model weights
checkpoint_path = "/content/resnetinceptionv1_epoch_32.pth"  # Update this path
checkpoint = torch.load(checkpoint_path, map_location=DEVICE)
model.load_state_dict(checkpoint)
print("Model weights loaded successfully.")

def create_montage(frames, size=(512, 512)):
    """Create a montage from selected frames."""
    montage = Image.new('RGB', size)
    num_images = len(frames)
    montage_grid = int(np.ceil(np.sqrt(num_images)))
    thumb_size = (size[0] // montage_grid, size[1] // montage_grid)

    for i, frame in enumerate(frames):
        thumbnail = frame.resize(thumb_size, Image.ANTIALIAS)
        x_offset = (i % montage_grid) * thumb_size[0]
        y_offset = (i // montage_grid) * thumb_size[1]
        montage.paste(thumbnail, (x_offset, y_offset))

    return montage

def predict(input_video):
    cap = cv2.VideoCapture(input_video)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    selected_frames = []
    frame_indices = np.linspace(0, total_frames - 1, 9, dtype=int)  # Select 9 frames

    for i in range(total_frames):
        ret, frame = cap.read()
        if not ret:
            break
        if i in frame_indices:
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            selected_frames.append(Image.fromarray(frame))

    cap.release()

    # Placeholder for deepfake detection logic
    # For demonstration purposes, we randomly select a classification
    video_label = "Fake" if np.random.rand() > 0.5 else "Real"
    fake_ratio = np.random.rand()
    detail = f"Placeholder deepfake ratio: {fake_ratio*100:.2f}%"

    montage = create_montage(selected_frames)

    return video_label, detail, montage

# Setup Gradio interface
interface = gr.Interface(
    fn=predict,
    inputs=gr.Video(label="Input Video"),
    outputs=[
        gr.Text(label="Classification"),
        gr.Text(label="Details"),
        gr.Image(label="Montage of Selected Frames")
    ],
).launch(debug=True, share=True)


RuntimeError: PytorchStreamReader failed reading zip archive: failed finding central directory